In [2]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 1.31 s (started: 2021-08-20 17:07:03 +08:00)


In [11]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 ch15 #1  Aug 20, 2021'

git push origin master

origin	git@github.com:ustchope/handson-ml2.git (fetch)
origin	git@github.com:ustchope/handson-ml2.git (push)
[master 77540b2] 更新 ch15 #1  Aug 19, 2021
 2 files changed, 663 insertions(+), 241 deletions(-)


To git@github.com:ustchope/handson-ml2.git
   5d5032d..77540b2  master -> master


time: 4.94 s (started: 2021-08-19 02:44:50 +08:00)


In [15]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 3.86 s (started: 2021-08-20 17:39:43 +08:00)


In [11]:
#初始化 Neptune 并创建新的运行

import neptune.new as neptune

run = neptune.init(project='ustchope/tf-keras-integration')

https://app.neptune.ai/ustchope/tf-keras-integration/e/TFKER-1
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
time: 6.95 s (started: 2021-08-20 17:24:40 +08:00)


In [ ]:
# 导入并实例化 NeptuneCallback 并将其传递给 model.fit() 的回调参数。
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')

# model.fit(x_train, y_train,
#           epochs=5,
#           batch_size=64,
#           callbacks=[neptune_cbk])


# 记录超参数，按照真实数据填入即可

PARAMS = {'lr':0.005, 
          'momentum':0.9, 
          'epochs':10,
          'batch_size':32}

# log hyper-parameters
run['hyper-parameters'] = PARAMS

# 使用 RNN 和 CNN 处理序列

击球手击球。外野手立即开始奔跑，预测球的轨迹。他跟踪它，调整他的动作，最后抓住它（在雷鸣般的掌声中）。预测未来是你一直在做的事情，无论你是在完成朋友的一句话，还是在期待早餐时咖啡的味道。在本章中，我们将讨论循环神经网络 (RNN)，这是一类可以预测未来的网络（当然，在一定程度上）。他们可以分析股票价格等时间序列数据，并告诉您何时买入或卖出。在自动驾驶系统中，他们可以预测汽车轨迹并帮助避免事故。更一般地说，它们可以处理任意长度的序列，而不是像我们目前考虑的所有网络那样处理固定大小的输入。例如，它们可以将句子、文档或音频样本作为输入，这使得它们对于自动翻译或语音到文本等自然语言处理应用非常有用。

在本章中，我们将首先了解 RNN 的基本概念以及如何使用时间反向传播来训练它们，然后我们将使用它们来预测时间序列。 之后我们将探讨 RNN 面临的两个主要困难：
* 不稳定梯度（在第 11 章中讨论），可以使用各种技术来缓解，包括循环 dropout 和循环层归一化
*（非常）有限的短期记忆，可以使用 LSTM 和 GRU 单元进行扩展

RNN 并不是唯一能够处理序列数据的神经网络类型：对于小序列，常规密集网络可以做到这一点； 对于非常长的序列，例如音频样本或文本，卷积神经网络实际上也可以很好地工作。 我们将讨论这两种可能性，我们将通过实现 WaveNet 来结束本章：这是一种 CNN 架构，能够处理数万个时间步长的序列。 在第 16 章中，我们将继续探索 RNN，看看如何将它们用于自然语言处理，以及基于注意力机制的最新架构。 让我们开始吧！

## 循环神经元和层

到目前为止，我们一直专注于前馈神经网络，其中激活仅在一个方向上流动，从输入层到输出层（附录 E 中讨论了一些例外）。 循环神经网络看起来非常像前馈神经网络，除了它也有指向后的连接。 让我们看一下最简单的 RNN，它由一个神经元接收输入、产生输出并将该输出发送回自身组成，如图 15-1（左）所示。 在每个时间步长 t（也称为帧），这个循环神经元接收输入 x(t) 以及它自己来自前一个时间步长 y(t-1) 的输出。 由于在第一个时间步没有以前的输出，所以一般设置为 0。我们可以将这个微小的网络表示在时间轴上，如图 15-1（右）所示。 这称为随时间展开网络（它是每个时间步表示一次的相同循环神经元）。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtld2yoy0kj61bk0ik76j02.jpg)

您可以轻松创建一层循环神经元。 在每个时间步 t，每个神经元都接收输入向量 x(t) 和来自前一个时间步 y(t–1) 的输出向量，如图 15-2 所示。 请注意，现在输入和输出都是向量（当只有一个神经元时，输出是标量）。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtld40uugyj61ca0jetbj02.jpg)

每个循环神经元都有两组权重：一组用于输入 x(t)，另一组用于前一时间步长 y(t-1) 的输出。 我们称这些权重向量为 $w_x$ 和 $w_y$。 如果我们考虑整个循环层而不是一个循环神经元，我们可以将所有权重向量放在两个权重矩阵 $W_x$ 和 $W_y$ 中。 然后，整个循环层的输出向量几乎可以按照您的预期进行计算，如公式 15-1 所示（b 是偏置向量，$φ(·)$ 是激活函数（例如 ReLU1）。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtld61bzc3j61ag06i74w02.jpg)

与前馈神经网络一样，我们可以通过将时间步 t 的所有输入放入输入矩阵 X(t) 中（参见公式 15-2），一次性计算整个小批量的循环层输出。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtld6wtb51j61b20cemya02.jpg)

在这个等式中：
* $\mathbf{Y}_{(t)}$ 是一个 $m \times n$ 神经元矩阵，其中包含小批量中每个实例在时间步长 $t$ 时层的输出（$m$ 是小批量中的实例数，$n$ 个神经元是神经元数）。
* $\mathbf{X}_{(t)}$ 是一个 $m \times n$ 输入矩阵，包含所有实例的输入（n 个输入是输入特征的数量）。
* $\mathbf{W}_x$ 是一个 $n_{inputs} \times n_{neurons}$ 矩阵，包含当前时间步长输入的连接权重。
* $\mathbf{W}_y$ 是一个神经元 $\times$ 神经元矩阵，包含前一时间步输出的连接权重。
* $b$ 是包含每个神经元偏置项的大小神经元向量。
* 权重矩阵 $\mathbf{W}_x$ 和 $\mathbf{W}_y$ 通常垂直串联成一个形状为 $(n_{inputs} + n_{neurons}) \times n_{neurons}$ 的权重矩阵 $\mathbf{W}$（参见公式 15-2 的第二行）。
* 符号$[\mathbf{X}_{(t)} \space \mathbf{Y}_{(t-1)}]$ 表示矩阵$\mathbf{X}_{(t)}$ 和$\mathbf{Y}_{(t-1)}$ 的水平串联。

请注意，$\mathbf{Y}_{(t)}$ 是 $\mathbf{X}_{(t)}$ 和 $\mathbf{Y}_{(t-1)}$ 的函数，它是 $\mathbf{X}_{(t-1)}$ 和 $\mathbf{Y}_{(t-2)}$ 的函数，它是 $\mathbf{X}_{(t- 2)}$ 和 $\mathbf{Y}_{(t–3)}$，依此类推。 这使得 $\mathbf{Y}_{(t)}$ 成为自时间 $t = 0$ 起所有输入的函数（即 $\mathbf{X}_{(0)}, \mathbf{X}_{(1)}, \cdots, \mathbf{X}_{(t)}$）。 在第一个时间步 $t = 0$，没有先前的输出，因此通常假设它们全为零。

### 记忆细胞

由于循环神经元在时间步 t 的输出是先前时间步的所有输入的函数，因此您可以说它具有某种形式的记忆。 跨时间步长保留某种状态的神经网络的一部分称为记忆单元（或简称为单元）。 单个循环神经元或循环神经元层是一个非常基本的细胞，只能学习短模式（通常长约 10 步，但这取决于任务）。 在本章的后面，我们将研究一些更复杂、更强大的细胞类型，它们能够学习更长的模式（大约长 10 倍，但这同样取决于任务）。

通常，单元格在时间步 $t$ 的状态，表示为 $\mathbf{h}_{(t)}$（“$\mathbf{h}$”代表“隐藏”），是该时间步的某些输入及其在前一时间步的状态的函数：$\mathbf{h}_{(t)} = f(\mathbf{h}_{(t–1)}, \mathbf{x}_{(t)})$。 它在时间步长 $t$ 的输出，表示为 $\mathbf{y}_{(t)}$ ，也是先前状态和当前输入的函数。 在我们目前讨论的基本单元的情况下，输出只是等于状态，但在更复杂的单元中情况并非总是如此，如图 15-3 所示。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtldydoq3lj61bu0juwg602.jpg)

### 输入和输出序列

RNN 可以同时接受一系列输入并产生一系列输出（参见图 15-4 中左上角的网络）。 这种类型的序列到序列网络对于预测诸如股票价格之类的时间序列非常有用：您将过去 N 天的价格提供给它，并且它必须输出未来一天的价格（即，从 N – 1天前到明天）。

或者，您可以为网络提供一系列输入并忽略除最后一个之外的所有输出（参见图 15-4 中右上角的网络）。 换句话说，这是一个序列到向量的网络。 例如，您可以向网络提供与电影评论相对应的单词序列，网络将输出情绪分数（例如，从 –1 [仇恨] 到 +1 [爱]）。

相反，您可以在每个时间步上一遍又一遍地向网络提供相同的输入向量，并让它输出一个序列（参见图 15-4 的左下角网络）。 这是一个向量到序列的网络。 例如，输入可以是图像（或 CNN 的输出），输出可以是该图像的标题。

最后，你可以有一个序列到向量网络，称为编码器，然后是一个向量到序列网络，称为解码器（见图 15-4 右下角的网络）。 例如，这可用于将句子从一种语言翻译成另一种语言。 你会向网络输入一种语言的句子，编码器将这个句子转换成单个向量表示，然后解码器将这个向量解码成另一种语言的句子。 这种称为编码器-解码器的两步模型比尝试使用单个序列到序列 RNN（如左上角表示的那个）即时翻译要好得多：句子的最后一个词可以影响 翻译的第一个词，所以你需要等到你看到整个句子再翻译。 我们将在第 16 章中看到如何实现编码器-解码器（正如我们将看到的，它比图 15-4 中建议的要复杂一些）。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtle1depv9j614c0t8jv202.jpg)

听起来很有希望，但是如何训练循环神经网络呢？

## 训练 RNN

要训练 RNN，诀窍是通过时间展开它（就像我们刚刚做的那样），然后简单地使用常规的反向传播（见图 15-5）。这种策略称为时间反向传播 (BPTT)。

就像在常规的反向传播中一样，有第一次前向通过展开的网络（由虚线箭头表示）。然后使用成本函数 $C(\mathbf{Y}_{(0)}, \mathbf{Y}_{(1)}, \cdots, \mathbf{Y}_{(T)})$ 评估输出序列)（其中 $T$ 是最大时间步长）。请注意，此成本函数可能会忽略某些输出，如图 15-5 所示（例如，在序列到向量 RNN 中，除了最后一个输出之外，所有输出都将被忽略）。然后，该成本函数的梯度通过展开的网络（由实线箭头表示）向后传播。最后，使用 BPTT 期间计算的梯度更新模型参数。请注意，梯度通过成本函数使用的所有输出向后流动，而不仅仅是通过最终输出（例如，在图 15-5 中，成本函数是使用网络的最后三个输出 $\mathbf{Y}_{(2)}$ 计算的， $\mathbf{Y}_{(3)}$ 和 $\mathbf{Y}_{(4)}$，所以梯度流经这三个输出，但不流经 $\mathbf{Y}_{(0)}$ 和 $\mathbf{Y}_{(1)}$)。此外，由于在每个时间步使用相同的参数 $\mathbf{W}$ 和 $\mathbf{b}$，反向传播将做正确的事情并在所有时间步上求和。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtle8jzjhvj614k0judhi02.jpg)

## 预测时间序列

假设您正在研究您网站上每小时的活跃用户数量，或您所在城市的每日温度，或您公司的财务状况，每季度使用多个指标进行测量。 在所有这些情况下，数据将是每个时间步长一个或多个值的序列。 这称为时间序列。 在前两个示例中，每个时间步长只有一个值，因此它们是单变量时间序列，而在财务示例中，每个时间步长有多个值（例如，公司的收入、债务等），因此是 多元时间序列。 一个典型的任务是预测未来值，这称为预测。 另一个常见的任务是填补空白：预测（或者说“postdict”）过去的缺失值。 这称为插补。 例如，图 15-6 显示了 3 个单变量时间序列，每个序列有 50 个时间步长，这里的目标是预测每个时间步长（由 X 表示）的值。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtleavei94j61440ecjtd02.jpg)

为简单起见，我们使用由 `generate_time_series()` 函数生成的时间序列，如下所示：

In [12]:
import numpy as np
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    return series[..., np.newaxis].astype(np.float32)

time: 2.56 ms (started: 2021-08-20 17:39:05 +08:00)


此函数根据请求创建尽可能多的时间序列（通过 batch_size 参数），每个序列的长度为 n_steps，并且每个序列中的每个时间步只有一个值（即，所有序列都是单变量）。 该函数返回一个形状为 [批量大小，时间步长，1] 的 NumPy 数组，其中每个序列是两个固定幅度但随机频率和相位的正弦波的总和，加上一点噪声。

> 在处理时间序列（以及其他类型的序列，例如句子）时，输入特征通常表示为形状 [批量大小、时间步长、维数] 的 3D 数组，其中对于单变量时间序列维数为 1，对于多变量时间维数更多 系列。

现在让我们使用这个函数创建一个训练集、一个验证集和一个测试集：

In [13]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

time: 95.5 ms (started: 2021-08-20 17:39:08 +08:00)


X_train 包含 7,000 个时间序列（即其形状为 [7000, 50, 1]），而 X_valid 包含 2,000 个（从第 7,000 个时间序列到第 8,999 个）和 X_test 包含 1,000 个（从第 9,000 个到第 9,999 个） . 由于我们想为每个系列预测单个值，因此目标是列向量（例如，y_train 的形状为 [7000, 1]）。

### 基线指标

在我们开始使用 RNN 之前，拥有一些基线指标通常是个好主意，否则我们最终可能会认为我们的模型效果很好，但实际上它比基本模型做得更差。 例如，最简单的方法是预测每个系列中的最后一个值。 这被称为天真的预测，有时要超越它是非常困难的。 在这种情况下，它给我们的均方误差约为 0.020：

In [16]:
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020136574

time: 5.28 s (started: 2021-08-20 17:39:53 +08:00)


另一种简单的方法是使用完全连接的网络。 由于每个输入都需要一个扁平的特征列表，我们需要添加一个 Flatten 层。 让我们用一个简单的线性回归模型使每个预测都是时间序列中的值：

In [17]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

time: 83.3 ms (started: 2021-08-20 17:42:27 +08:00)


如果我们使用 MSE 损失和默认的 Adam 优化器编译这个模型，然后将其拟合到训练集上 20 个时期并在验证集上对其进行评估，我们会得到大约 0.004 的 MSE。 这比幼稚的方法要好得多！

### 实现一个简单的 RNN

让我们看看我们是否可以用一个简单的 RNN 打败它：

In [18]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

time: 129 ms (started: 2021-08-20 17:44:04 +08:00)


这确实是您可以构建的最简单的 RNN。正如我们在图 15-1 中看到的那样，它只包含一个层和一个神经元。我们不需要指定输入序列的长度（与之前的模型不同），因为循环神经网络可以处理任意数量的时间步长（这就是我们将第一个输入维度设置为 None 的原因）。默认情况下，SimpleRNN 层使用双曲正切激活函数。它的工作原理与我们之前看到的完全一样：初始状态 h(init) 设置为 0，并将它与第一个时间步的值 x(0) 一起传递给单个循环神经元。神经元计算这些值的加权和，并将双曲正切激活函数应用于结果，这给出了第一个输出 y0。在一个简单的 RNN 中，这个输出也是新状态 h0。这个新状态与下一个输入值 x(1) 一起传递给同一个循环神经元，并重复该过程直到最后一个时间步。然后该层只输出最后一个值，y49。所有这些都是针对每个时间序列同时执行的。

> 默认情况下，Keras 中的循环层仅返回最终输出。 为了让它们在每个时间步返回一个输出，您必须设置 return_sequences=True，正如我们将看到的。

如果你编译、拟合和评估这个模型（就像之前，我们使用 Adam 训练了 20 个 epochs），你会发现它的 MSE 只达到 0.014，所以它比朴素的方法好，但它没有击败简单的线性 模型。 请注意，对于每个神经元，线性模型的每个输入和每个时间步都有一个参数，加上一个偏置项（在我们使用的简单线性模型中，总共有 51 个参数）。 相比之下，对于简单 RNN 中的每个循环神经元，每个输入和每个隐藏状态维度只有一个参数（在简单 RNN 中，这只是层中循环神经元的数量），加上一个偏置项。 在这个简单的 RNN 中，总共只有三个参数。

**趋势和季节性**

---

还有许多其他模型可以预测时间序列，例如加权移动平均模型或自回归积分移动平均 (ARIMA) 模型。其中一些要求您首先删除趋势和季节性。例如，如果您正在研究网站上的活跃用户数量，并且每月增长 10%，则您必须从时间序列中删除此趋势。一旦模型经过训练并开始进行预测，您就必须重新添加趋势以获得最终预测。同样，如果您试图预测每个月的防晒乳液销量，您可能会观察到很强的季节性：因为它每年夏天都卖得很好，所以每年都会重复类似的模式。您必须从时间序列中去除这种季节性，例如通过计算每个时间步的值与一年前的值之间的差异（这种技术称为差分）。同样，在模型经过训练并做出预测后，您必须重新添加季节性模式以获得最终预测。

使用 RNN 时，通常没有必要做所有这些，但在某些情况下它可能会提高性能，因为模型不必学习趋势或季节性。

---

显然，我们的简单 RNN 太简单了，无法获得良好的性能。 所以让我们尝试添加更多的循环层！

### 深度 RNN

堆叠多层单元是很常见的，如图 15-7 所示。 这为您提供了深度 RNN。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtnem0u4dnj61420js76t02.jpg)

使用 tf.keras 实现深度 RNN 非常简单：只需堆叠循环层。 在这个例子中，我们使用了三个 SimpleRNN 层（但我们可以添加任何其他类型的循环层，例如 LSTM 层或 GRU 层，我们将在稍后讨论）：

In [19]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

time: 208 ms (started: 2021-08-20 17:57:06 +08:00)


> 确保为所有循环层设置 return_sequences=True （除了最后一个，如果你只关心最后一个输出）。 如果你不这样做，他们将输出一个 2D 数组（只包含最后一个时间步的输出）而不是一个 3D 数组（包含所有时间步的输出），并且下一个循环层会抱怨你没有喂它 预期 3D 格式的序列。

如果您编译、拟合和评估此模型，您会发现它达到了 0.003 的 MSE。我们终于成功击败了线性模型！

请注意，最后一层并不理想：它必须有一个单位，因为我们要预测单变量时间序列，这意味着每个时间步长必须有一个输出值。但是，具有单个单元意味着隐藏状态只是一个数字。这真的不多，而且可能没那么有用；据推测，RNN 将主要使用其他循环层的隐藏状态来携带它在时间步到时间步所需的所有信息，并且不会非常多地使用最后一层的隐藏状态。此外，由于 SimpleRNN 层默认使用 tanh 激活函数，因此预测值必须在 –1 到 1 的范围内。但是如果您想使用另一个激活函数怎么办？由于这两个原因，最好用 Dense 层替换输出层：它会运行得稍微快一点，准确率大致相同，并且允许我们选择我们想要的任何输出激活函数。如果您进行此更改，还要确保从第二个（现在是最后一个）循环层中删除 return_sequences=True ：

In [20]:
model = keras.models.Sequential([
keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
keras.layers.SimpleRNN(20),
keras.layers.Dense(1)
])

time: 157 ms (started: 2021-08-20 18:02:17 +08:00)


如果你训练这个模型，你会发现它收敛得更快，性能也一样好。 另外，您可以根据需要更改输出激活函数。

### 提前预测几个时间步

到目前为止，我们只预测了下一个时间步的值，但是我们可以通过适当地更改目标（例如，要预测提前 10 步，只需将目标值更改为 10 领先一步而不是领先一步）。 但是如果我们想预测接下来的 10 个值呢？

第一种选择是使用我们已经训练过的模型，让它预测下一个值，然后将该值添加到输入中（就像这个预测值确实发生过一样），然后再次使用该模型来预测下一个值，并且 依此类推，如以下代码所示：

In [ ]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new

for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)
Y_pred = X[:, n_steps:]

正如您所料，下一步的预测通常比后续时间步的预测更准确，因为错误可能会累积（如图 15-8 所示）。 如果您在验证集上评估此方法，您会发现 MSE 约为 0.029。 这比之前的模型要高得多，但也是一项艰巨的任务，因此比较没有多大意义。 将此性能与朴素预测（仅预测时间序列将在 10 个时间步长内保持不变）或与简单的线性模型进行比较更有意义。 天真的方法很糟糕（它给出的 MSE 约为 0.223），但线性模型给出的 MSE 约为 0.0188：它比使用我们的 RNN 一步一步地预测未来要好得多，而且训练和训练速度也快得多。 跑。 不过，如果您只想提前预测几个时间步，在更复杂的任务上，这种方法可能会很有效。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtnewx8zqaj61fc0qg41p02.jpg)

第二种选择是训练 RNN 一次预测所有 10 个下一个值。 我们仍然可以使用序列到向量模型，但它会输出 10 个值而不是 1。但是，我们首先需要将目标更改为包含接下来 10 个值的向量：

In [21]:
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

time: 99.7 ms (started: 2021-08-20 18:11:48 +08:00)


现在我们只需要输出层有 10 个单元而不是 1 个：

In [22]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

time: 158 ms (started: 2021-08-20 18:13:24 +08:00)


训练此模型后，您可以非常轻松地一次预测接下来的 10 个值：

In [ ]:
Y_pred = model.predict(X_new)

该模型运行良好：接下来 10 个时间步的 MSE 约为 0.008。 这比线性模型好得多。 但是我们仍然可以做得更好：实际上，我们可以训练模型在每个时间步预测接下来的 10 个值，而不是仅在最后一个时间步训练模型来预测接下来的 10 个值。 换句话说，我们可以把这个序列到向量的 RNN 变成一个序列到序列的 RNN。 这种技术的优点是损失将包含每个时间步的 RNN 输出项，而不仅仅是最后一个时间步的输出。 这意味着将有更多的误差梯度流经模型，而且它们不必只流经时间； 它们也将从每个时间步的输出中流出。 这将稳定并加速训练。

需要明确的是，在时间步长 0 时，模型将输出包含时间步长 1 到 10 的预测的向量，然后在时间步长 1 时，模型将预测时间步长 2 到 11，依此类推。 所以每个目标必须是一个与输入序列长度相同的序列，每一步都包含一个10维的向量。 让我们准备这些目标序列：

In [ ]:
Y = np.empty((10000, n_steps, 10)) # each target is a sequence of 10D vectors
for step_ahead in range(1, 10 + 1):
    Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

> 目标将包含出现在输入中的值可能令人惊讶（X_train 和 Y_train 之间有很多重叠）。 这不是作弊吗？ 幸运的是，一点也不：在每个时间步，模型只知道过去的时间步，所以它不能向前看。 据说这是一个因果模型。

要将模型变成序列到序列模型，我们必须在所有循环层（甚至最后一层）中设置 return_sequences=True，并且我们必须在每个时间步应用输出 Dense 层。 Keras 为此提供了一个 TimeDistributed 层：它包装任何层（例如，密集层）并在其输入序列的每个时间步长处应用它。它通过重塑输入来有效地做到这一点，以便每个时间步都被视为一个单独的实例（即，它将输入从 [批量大小、时间步长、输入维度] 重塑为 [批量大小 × 时间步长、输入维度]；在这个例子中，输入维数为 20，因为前面的 SimpleRNN 层有 20 个单元），然后运行 ​​Dense 层，最后将输出重新整形回序列（即，将输出从 [batch size × time步骤，输出维度]到[批量大小，时间步长，输出维度]；在这个例子中，输出维度的数量是10，因为密集层有10个单位）。这是更新后的模型：

In [ ]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

Dense 层实际上支持序列作为输入（甚至更高维度的输入）：它像 TimeDistributed(Dense(...)) 一样处理它们，这意味着它只应用于最后一个输入维度（独立于所有时间步）。 因此，我们可以只用 Dense(10) 替换最后一层。 然而，为了清楚起见，我们将继续使用 TimeDistributed(Dense(10))，因为它清楚地表明 Dense 层在每个时间步独立应用，并且模型将输出一个序列，而不仅仅是单个向量。

训练期间需要所有输出，但只有最后一个时间步的输出对预测和评估有用。 因此，尽管我们将依赖于所有输出的 MSE 进行训练，但我们将使用自定义指标进行评估，只计算最后一个时间步输出的 MSE：

In [24]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss="mse", optimizer=optimizer, metrics=[last_time_step_mse])

time: 70.3 ms (started: 2021-08-20 19:02:28 +08:00)


/public/huangwei/miniconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


我们得到了大约 0.006 的验证 MSE，比之前的模型好 25%。 您可以将此方法与第一种方法结合使用：只需使用此 RNN 预测接下来的 10 个值，然后将这些值连接到输入时间序列并再次使用模型预测接下来的 10 个值，并根据需要重复该过程多次 . 使用这种方法，您可以生成任意长的序列。 对于长期预测，它可能不是很准确，但如果您的目标是生成原创音乐或文本，这可能很好，我们将在第 16 章中看到。

在预测时间序列时，在您的预测中包含一些误差线通常很有用。 为此，一种有效的技术是 MC Dropout，在第 11 章介绍：在每个存储单元中添加一个 MC Dropout 层，丢弃部分输入和隐藏状态。 训练后，要预测新的时间序列，请多次使用该模型并计算每个时间步预测的均值和标准差。

简单的 RNN 可以很好地预测时间序列或处理其他类型的序列，但它们在长时间序列或序列上表现不佳。 让我们讨论原因，看看我们能做些什么。

## 处理长序列

要在长序列上训练 RNN，我们必须在许多时间步上运行它，使展开的 RNN 成为一个非常深的网络。 就像任何深度神经网络一样，它可能会遇到梯度不稳定问题，这在第 11 章中讨论过：训练可能需要很长时间，或者训练可能不稳定。 此外，当 RNN 处理一个长序列时，它会逐渐忘记序列中的第一个输入。 让我们看看这两个问题，从不稳定梯度问题开始。

### 解决不稳定梯度问题

我们在深度网络中用于缓解不稳定梯度问题的许多技巧也可以用于 RNN：良好的参数初始化、更快的优化器、dropout 等。然而，非饱和激活函数（例如 ReLU）在这里可能没有多大帮助；事实上，它们实际上可能导致 RNN 在训练过程中更加不稳定。为什么？好吧，假设梯度下降以一种在第一时间步长略微增加输出的方式更新权重。因为在每个时间步都使用相同的权重，所以第二个时间步的输出也可能会略微增加，第三步的输出，依此类推，直到输出爆炸——非饱和激活函数并不能阻止这一点。您可以通过使用较小的学习率来降低这种风险，但您也可以简单地使用像双曲正切这样的饱和激活函数（这解释了为什么它是默认值）。以同样的方式，梯度本身可以爆炸。如果您注意到训练不稳定，您可能需要监控梯度的大小（例如，使用 TensorBoard）并可能使用 Gradient Clipping。

此外，RNN 不能像深度前馈网络那样有效地使用批归一化。事实上，你不能在时间步之间使用它，只能在循环层之间使用。更准确地说，在技术上可以将 BN 层添加到存储单元（我们将很快看到），以便在每个时间步（在该时间步的输入和隐藏状态）上应用它上一步）。但是，无论输入和隐藏状态的实际规模和偏移如何，每个时间步都将使用相同的 BN 层，具有相同的参数。在实践中，这不会产生好的结果，正如 César Laurent 等人所证明的那样。在 2015 年的一篇论文中：作者发现 BN 仅在应用于输入时略微有益，而不是应用于隐藏状态。换句话说，当应用在循环层之间（即图 15-7 中的垂直），而不是循环层内（即水平）时，它总比没有好。在 Keras 中，这可以通过在每个循环层之前添加一个 Batch Normalization 层来完成，但不要期望太多。

另一种形式的归一化通常更适合 RNN：层归一化。 这个想法是由 Jimmy Lei Ba 等人提出的。 在 2016 年的一篇论文中：它与 Batch Normalization 非常相似，但它不是跨批处理维度进行归一化，而是跨特征维度进行归一化。 一个优点是它可以在每个时间步独立地为每个实例动态计算所需的统计数据。 这也意味着它在训练和测试期间的行为方式相同（与 BN 不同），并且不需要使用指数移动平均值来估计训练集中所有实例的特征统计数据。 与 BN 一样，Layer Normalization 为每个输入学习一个尺度和一个偏移参数。 在 RNN 中，它通常在输入和隐藏状态的线性组合之后立即使用。

让我们使用 tf.keras 在一个简单的内存单元中实现层归一化。 为此，我们需要定义一个自定义内存单元。 它就像一个普通层，除了它的 call() 方法有两个参数：当前时间步的输入和前一时间步的隐藏状态。 请注意， states 参数是一个包含一个或多个张量的列表。 在一个简单的 RNN 单元的情况下，它包含一个等于前一个时间步的输出的张量，但其他单元可能有多个状态张量（例如，LSTMCell 有一个长期状态和一个短期状态，因为我们 很快就会看到）。 单元格还必须具有 state_size 属性和 output_size 属性。 在简单的 RNN 中，两者都简单地等于单元数。 以下代码实现了一个自定义内存单元，它的行为类似于 SimpleRNNCell，除了它还会在每个时间步应用层归一化：

In [26]:
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,
                                                          activation=None)
        self.layer_norm = keras.layers.LayerNormalization()
        self.activation = keras.activations.get(activation)

    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

time: 1.97 ms (started: 2021-08-20 19:10:55 +08:00)


代码非常简单。我们的 LNSimpleRNNCell 类继承自 keras.layers.Layer 类，就像任何自定义层一样。构造函数获取单元数和所需的激活函数，并设置 state_size 和 output_size 属性，然后创建一个没有激活函数的 SimpleRNNCell（因为我们希望在线性操作之后但在激活函数之前执行层归一化）。然后构造函数创建 LayerNormalization 层，最后它获取所需的激活函数。 call() 方法首先应用简单的 RNN 单元，它计算当前输入和先前隐藏状态的线性组合，并返回两次结果（实际上，在 SimpleRNNCell 中，输出刚好等于隐藏状态） ：换句话说，new_states[0] 等于输出，因此我们可以安全地忽略 call() 方法其余部分中的 new_states。接下来， call() 方法应用层归一化，然后是激活函数。最后，它返回两次输出（一次作为输出，一次作为新的隐藏状态）。要使用这个自定义单元格，我们需要做的就是创建一个 keras.layers.RNN 层，并传递一个单元格实例：

In [ ]:
model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True, input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

同样，您可以创建一个自定义单元格以在每个时间步之间应用 dropout。 但是有一个更简单的方法：所有循环层（除了 keras.layers.RNN）和 Keras 提供的所有单元格都有一个 dropout 超参数和一个 recurrent_dropout 超参数：前者定义了应用于输入的 dropout 率（在每个时间步） ，后者定义了隐藏状态的丢失率（也在每个时间步）。 无需创建自定义单元来在 RNN 中的每个时间步应用 dropout。

使用这些技术，您可以缓解不稳定梯度问题并更有效地训练 RNN。 现在让我们看看如何处理短期记忆问题。

### 解决短期记忆问题

由于数据在遍历 RNN 时经历的转换，每个时间步都会丢失一些信息。 一段时间后，RNN 的状态几乎不包含第一个输入的痕迹。 这可能是一个亮点。 想象一下小鱼多莉试图翻译一个长句； 当她读完它时，她不知道它是如何开始的。 为了解决这个问题，已经引入了具有长期记忆的各种类型的细胞。 事实证明，它们非常成功，以至于基本电池不再使用太多。 让我们首先看看这些长期记忆单元中最受欢迎的：LSTM 单元。

#### LSTM 单元

长短期记忆 (LSTM) 单元是由 Sepp Hochreiter 和 Jürgen Schmidhuber 于 1997 年提出的，并在多年来逐渐得到了几位研究人员的改进，例如 Alex Graves、Haşim Sak 和 Wojciech Zaremba。 如果将 LSTM 单元视为黑盒，则它可以非常像基本单元那样使用，只是性能会好得多； 训练会更快收敛，并且会检测数据中的长期依赖关系。 在 Keras 中，您可以简单地使用 LSTM 层而不是 SimpleRNN 层：

In [28]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

time: 440 ms (started: 2021-08-20 19:24:35 +08:00)


或者，您可以使用通用的 keras.layers.RNN 层，将 LSTMCell 作为参数：

In [30]:
model = keras.models.Sequential([
    keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True, input_shape=[None, 1]),
    keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

time: 183 ms (started: 2021-08-20 19:25:10 +08:00)


但是，LSTM 层在 GPU 上运行时使用了优化的实现（参见第 19 章），因此通常最好使用它（RNN 层在定义自定义单元时最有用，就像我们之前所做的那样）。

那么 LSTM 单元是如何工作的呢？ 其架构如图15-9所示。

如果你不看盒子里面的东西，LSTM 单元看起来就像一个普通的单元，除了它的状态被分成两个向量：h(t) 和 c(t)（“c”代表“单元” ）。 您可以将 h(t) 视为短期状态，将 c(t) 视为长期状态。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtnh8xmw9oj61360pm0va02.jpg)

现在让我们打开盒子！ 关键思想是网络可以学习在长期状态下存储什么、丢弃什么以及从中读取什么。 当长期状态 c(t-1) 从左到右遍历网络时，可以看到它首先通过一个遗忘门，丢弃一些记忆，然后通过加法运算添加一些新的记忆（增加 由输入门选择的记忆）。 结果 c(t) 直接发送出去，没有任何进一步的变换。 因此，在每个时间步长中，都会删除一些记忆并添加一些记忆。 而且，在加法运算之后，长期状态被复制并通过tanh函数，然后结果被输出门过滤。 这会产生短期状态 h(t)（它等于该时间步长中单元格的输出 y(t)）。 现在让我们看看新记忆从何而来以及门是如何工作的。

首先，当前输入向量 x(t) 和之前的短期状态 h(t-1) 被馈送到四个不同的全连接层。 它们都有不同的用途：
* 主层是输出 g(t) 的层。 它通常具有分析当前输入 x(t) 和先前（短期）状态 h(t-1) 的作用。 在基本单元格中，除了这一层之外别无其他，其输出直接输出到 y(t) 和 h(t) 。 相比之下，在 LSTM 单元中，该层的输出不会直接输出，而是将其最重要的部分存储在长期状态中（其余部分被丢弃）。
* 其他三层是门控制器。 由于它们使用逻辑激活函数，因此它们的输出范围从 0 到 1。如您所见，它们的输出被馈送到逐元素乘法运算，因此如果它们输出 0，则关闭门，如果输出 1，则打开门 . 具体来说：
    - 遗忘门（由 f(t) 控制）控制应擦除长期状态的哪些部分。
    - 输入门（由 i(t) 控制）控制应将 g(t) 的哪些部分添加到长期状态。
    - 最后，输出门（由 o(t) 控制）控制应该在这个时间步读取和输出长期状态的哪些部分，包括 h(t) 和 y(t)。

简而言之，LSTM 单元可以学习识别重要输入（这就是输入门的作用），将其存储在长期状态中，并在需要时保留它（这就是遗忘门的作用） ，并在需要时提取它。 这就解释了为什么这些细胞在捕捉时间序列、长文本、录音等的长期模式方面取得了惊人的成功。

公式 15-3 总结了如何为单个实例计算单元格的长期状态、短期状态和输出（整个小批量的公式非常相似）。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtnhdytpoyj61ge0l6wgr02.jpg)

在这个等式中：
* $\mathbf{W}_{xi}、\mathbf{W}_{xf}、\mathbf{W}_{xo}、\mathbf{W}_{xg}$ 是四个层中每一层的权重矩阵，用于连接到输入向量 $x(t)$。
* $\mathbf{W}_{hi}、\mathbf{W}_{hf}、\mathbf{W}_{ho}$ 和 $\mathbf{W}_{hg}$ 是四个层中每一层的权重矩阵，用于连接到先前的短期状态 $h(t-1)$。
* $b_{i} 、 b_{f} 、 b_{o}$ 和 $b_{g}$ 是四层中每一层的偏置项。 请注意，TensorFlow 将 $b_f$ 初始化为一个全为 1 而不是 0 的向量。 这可以防止在训练开始时忘记一切。

#### 窥孔连接

在常规 LSTM 单元中，门控制器只能查看输入 x(t) 和之前的短期状态 h(t–1)。 通过让他们同时查看长期状态，为他们提供更多背景信息可能是个好主意。 这个想法是由 Felix Gers 和 Jürgen Schmidhuber 在 2000 年提出的。 他们提出了一种带有额外连接的 LSTM 变体，称为窥视孔连接：先前的长期状态 c(t-1) 作为输入添加到遗忘门的控制器和 输入门，当前的长期状态 c(t) 作为输入添加到输出门的控制器。 这通常会提高性能，但并非总是如此，并且没有明确的模式来说明哪些任务在有或没有它们的情况下更好：你必须在你的任务上尝试它，看看它是否有帮助。

在 Keras 中，LSTM 层基于 keras.layers.LSTMCell 单元，不支持窥视孔。 然而，实验性 tf.keras.experimental.PeepholeLSTMCell 确实如此，因此您可以创建一个 keras.layers.RNN 层并将 PeepholeLSTM Cell 传递给其构造函数。

LSTM 单元还有许多其他变体。 一种特别流行的变体是 GRU 单元，我们现在将介绍它。

#### GRU单元

门控循环单元 (GRU) 单元（见图 15-10）由 Kyunghyun Cho 等人提出。 在 2014 年的一篇论文中也介绍了我们之前讨论过的编码器-解码器网络。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtnhlo50d5j61260qumyz02.jpg)

GRU 单元是 LSTM 单元的简化版本，它的性能似乎也一样（这解释了它日益流行的原因）。 这些是主要的简化：
* 两个状态向量合并为一个向量h(t)。
* 单个门控制器 z(t) 控制遗忘门和输入门。 如果门控制器输出 1，则遗忘门打开 (= 1)，输入门关闭 (1 – 1 = 0)。 如果它输出 0，则会发生相反的情况。 换句话说，无论何时必须存储内存，都会首先擦除将要存储的位置。 这实际上是 LSTM 单元本身的常见变体。
* 没有输出门； 在每个时间步输出完整的状态向量。 然而，有一个新的门控制器 r(t) 控制前一状态的哪一部分将显示给主层 (g(t) )。

公式 15-4 总结了如何为单个实例计算每个时间步长的单元格状态。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtni0gar6gj61gi0emjt102.jpg)

Keras 提供了 keras.layers.GRU 层（基于 keras.layers.GRUCell 存储单元）； 使用它只是用 GRU 替换 SimpleRNN 或 LSTM 的问题。

LSTM 和 GRU 单元是 RNN 成功背后的主要原因之一。 然而，尽管它们可以处理比简单 RNN 长得多的序列，但它们的短期记忆仍然相当有限，而且它们很难在 100 个时间步长或更多的序列中学习长期模式，例如音频样本、长时间 系列，或长句。 解决这个问题的一种方法是缩短输入序列，例如使用一维卷积层。

#### 使用一维卷积层处理序列

在第 14 章中，我们看到 2D 卷积层通过在图像上滑动几个相当小的内核（或过滤器）来工作，产生多个 2D 特征图（每个内核一个）。类似地，一维卷积层在一个序列上滑动多个内核，为每个内核生成一个一维特征图。每个内核将学习检测单个非常短的序列模式（不超过内核大小）。如果使用 10 个内核，则该层的输出将由 10 个一维序列（所有长度相同）组成，或者等效地，您可以将此输出视为单个 10 维序列。这意味着您可以构建一个由循环层和一维卷积层（甚至一维池化层）混合组成的神经网络。如果使用步长为 1 且填充“相同”的一维卷积层，则输出序列的长度将与输入序列相同。但是，如果您使用“有效”填充或步长大于 1，则输出序列将比输入序列短，因此请确保相应地调整目标。例如，下面的模型与之前的模型相同，只是它从一个 1D 卷积层开始，将输入序列下采样因子 2，使用步长 2。内核大小大于步长，因此所有输入都将用于计算层的输出，因此模型可以学习保留有用的信息，只删除不重要的细节。通过缩短序列，卷积层可以帮助 GRU 层检测更长的模式。请注意，我们还必须裁剪目标中的前三个时间步长（由于内核的大小为 4，卷积层的第一个输出将基于输入时间步长 0 到 3），并按一个因子对目标进行下采样共 2 个：

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
    input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train[:, 3::2], epochs=20,
validation_data=(X_valid, Y_valid[:, 3::2]))

如果你训练和评估这个模型，你会发现它是迄今为止最好的模型。 卷积层确实有帮助。 事实上，实际上可以只使用一维卷积层并完全丢弃循环层！

#### WaveNet

在 2016 年的一篇论文中，Aaron van den Oord 和其他 DeepMind 研究人员介绍了一种名为 WaveNet 的架构。 他们堆叠了 1D 卷积层，在每一层将扩张率（每个神经元输入的分散程度）加倍：第一个卷积层一次只看到两个时间步长，而下一个看到四个时间步长（它的接受 字段是四个时间步长），下一个看到八个时间步长，依此类推（见图 15-11）。 这样，较低层学习短期模式，而较高层学习长期模式。 由于膨胀率加倍，网络可以非常有效地处理极大的序列。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtnidaakonj61420hwq7d02.jpg)

1, 2, 4, 8, ..., 256, 512，然后他们堆叠另一组 10 个相同的层（同样具有膨胀率 1, 2, 4, 8, ..., 256, 512），然后又是另一个相同的层 10层一组。 他们通过指出具有这些膨胀率的 10 个卷积层的单个堆栈将充当内核大小为 1,024 的超高效卷积层（除了速度更快、功能更强大且使用明显更少的参数之外）来证明这种架构的合理性，其中 这就是他们堆叠 3 个这样的块的原因。 他们还在每一层之前用等于扩张率的多个零填充输入序列，以在整个网络中保持相同的序列长度。 以下是如何实现一个简化的 WaveNet 来处理与之前相同的序列：

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))

for rate in (1, 2, 4, 8) * 2:
    model.add(keras.layers.Conv1D(filters=20, 
                                  kernel_size=2, 
                                  padding="causal",
                                  activation="relu", 
                                  dilation_rate=rate))

model.add(keras.layers.Conv1D(filters=10, kernel_size=1))

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])

history = model.fit(X_train, Y_train, epochs=20,
validation_data=(X_valid, Y_valid))

这个 Sequential 模型从一个显式的输入层开始（这比尝试只在第一层设置 input_shape 更简单），然后继续使用“因果”填充的一维卷积层：这确保了卷积层不会窥视未来 在进行预测时（相当于在左侧用适量的零填充输入并使用“有效”填充）。 然后，我们使用不断增长的膨胀率添加类似的层对：1, 2, 4, 8，再次是 1, 2, 4, 8。 最后，我们添加输出层：具有 10 个大小为 1 的过滤器的卷积层，没有任何过滤器 激活函数。 由于填充层，每个卷积层输出一个与输入序列长度相同的序列，因此我们在训练过程中使用的目标可以是完整的序列：无需裁剪它们或对它们进行下采样。

到目前为止，最后两个模型在预测我们的时间序列方面提供了最佳性能！ 在 WaveNet 论文中，作者在各种音频任务（因此架构的名称）上取得了最先进的性能，包括文本到语音任务，在多种语言中产生令人难以置信的逼真的声音。 他们还使用该模型生成音乐，一次一个音频样本。 当您意识到一秒钟的音频可以包含数万个时间步时，这一壮举就更加令人印象深刻——即使 LSTM 和 GRU 也无法处理如此长的序列。

在第 16 章中，我们将继续探索 RNN，我们将看到它们如何处理各种 NLP 任务。